In [2]:
import sqlite3
import csv
from pprint import pprint

In [90]:
db = sqlite3.connect("sheepshead.db")
c = db.cursor()

In [98]:
#Create nodes table
c.execute('DROP TABLE IF EXISTS nodes')
QUERY = '''CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);'''
c.execute(QUERY)
db.commit()
idList = []
count = 0
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    #for i in dr:    
    to_db = [(i['id'], i['lat'],i['lon'], i['user'].decode("utf-8"), 
              i['uid'], i['version'], i['changeset'], 
              i['timestamp']) for i in dr]
    '''if i['id'] in idList:
        print i['id']
    else:
        idList.append(i['id'])'''

In [99]:
# insert the formatted data?
c.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
# commit the changes
db.commit()

In [18]:
#Create nodes_tags table
QUERY = '''CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);'''
c.execute(QUERY)
db.commit()
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"),i['value'].decode("utf-8"),
              i['type'].decode("utf-8")) for i in dr]

In [19]:
# insert the formatted data?
c.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
# commit the changes
db.commit()

In [21]:
#Create ways table
QUERY = '''CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);'''
c.execute(QUERY)
db.commit()
with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['user'].decode("utf-8"),i['uid'].decode("utf-8"),
              i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]

In [22]:
# insert the formatted data?
c.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
# commit the changes
db.commit()

In [24]:
#Create ways_tags table
QUERY = '''CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);'''
c.execute(QUERY)
db.commit()
with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"),i['value'].decode("utf-8"),
              i['type'].decode("utf-8")) for i in dr]

In [25]:
# insert the formatted data?
c.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
# commit the changes
db.commit()

In [27]:
#Create ways_nodes table
QUERY = '''CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);'''
c.execute(QUERY)
db.commit()
with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) for i in dr]

In [28]:
# insert the formatted data?
c.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db)
# commit the changes
db.commit()

In [100]:
#Number of nodes
QUERY = "SELECT count(*) FROM nodes;"
c.execute(QUERY)
rows = c.fetchall()
pprint(rows)

[(233024,)]


In [103]:
#Number of nodes_tags
QUERY = "SELECT count(*) FROM nodes_tags;"
c.execute(QUERY)
rows = c.fetchall()
pprint(rows)

[(16381,)]


In [104]:
#Number of ways_tags
QUERY = "SELECT count(*) FROM ways_tags;"
c.execute(QUERY)
rows = c.fetchall()
pprint(rows)

[(248458,)]


In [105]:
#Number of ways
QUERY = "SELECT count(*) FROM ways;"
c.execute(QUERY)
rows = c.fetchall()
pprint(rows)

[(48981,)]


In [106]:
#Number of ways_nodes
QUERY = "SELECT count(*) FROM ways_nodes;"
c.execute(QUERY)
rows = c.fetchall()
pprint(rows)

[(317939,)]


In [115]:
#Number of unique users
QUERY = "SELECT count(DISTINCT(uid)) from (select uid FROM ways union all select DISTINCT uid from nodes);"
c.execute(QUERY)
users = c.fetchall()
pprint(users)

[(129,)]


In [114]:
#Top ten users
QUERY = '''SELECT user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways)
GROUP BY user
ORDER BY num DESC
LIMIT 10;'''
c.execute(QUERY)
topusers = c.fetchall()
pprint(topusers)

[(u'Rub21_nycbuildings', 244623),
 (u'ingalls', 15758),
 (u'celosia_nycbuildings', 4501),
 (u'ingalls_nycbuildings', 3339),
 (u'aaron_nycbuildings', 2434),
 (u'watame', 1782),
 (u'ediyes_nycbuildings', 1552),
 (u'rjhale1971', 1255),
 (u'woodpeck_fixbot', 1011),
 (u'MxxCon', 884)]


In [121]:
#Amenities
QUERY = '''SELECT value, COUNT(*) as num
FROM (SELECT value,key FROM nodes_tags UNION ALL SELECT value,key FROM ways_tags)
WHERE key='amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 10;'''
c.execute(QUERY)
amenity = c.fetchall()
pprint(amenity)

[(u'parking', 135),
 (u'school', 63),
 (u'bench', 59),
 (u'place_of_worship', 57),
 (u'bicycle_parking', 34),
 (u'bank', 14),
 (u'fire_station', 11),
 (u'fuel', 10),
 (u'pharmacy', 10),
 (u'restaurant', 7)]


In [162]:
#Top religion for place of worship
QUERY = '''SELECT l.value, l.count1 +l.count2 as num
from ((SELECT nodes_tags.value, COUNT(*) as count1
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') as i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value) as j 
JOIN (SELECT ways_tags.value, COUNT(*) as count2
FROM ways_tags 
    JOIN (SELECT DISTINCT(id) FROM ways_tags WHERE value='place_of_worship') as k
    ON ways_tags.id=k.id
WHERE ways_tags.key='religion'
GROUP BY ways_tags.value) as m 
on j.value = m.value) as l
group by l.value
order by num desc
limit 1;
'''
c.execute(QUERY)
religion = c.fetchall()
pprint(religion)


[(u'jewish', 35)]


In [167]:
#Streets with most data
QUERY = '''SELECT value, COUNT(*) as num
FROM (SELECT value,key FROM nodes_tags UNION ALL SELECT value,key FROM ways_tags)
WHERE key='street'
GROUP BY value
ORDER BY num DESC
LIMIT 10;'''
c.execute(QUERY)
street = c.fetchall()
pprint(street)

[(u'East 28th Street', 755),
 (u'East 26th Street', 751),
 (u'East 29th Street', 715),
 (u'Bedford Avenue', 674),
 (u'East 27th Street', 648),
 (u'East 23rd Street', 626),
 (u'East 21st Street', 612),
 (u'East 22nd Street', 600),
 (u'Coney Island Avenue', 593),
 (u'East 15th Street', 585)]


In [171]:
#Average height of building
QUERY = '''SELECT (sum(value)/COUNT(*)) as num
FROM (SELECT value,key FROM nodes_tags UNION ALL SELECT value,key FROM ways_tags)
WHERE key='height';'''
c.execute(QUERY)
height = c.fetchall()
pprint(height)

[(7.240978514108155,)]
